In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import re

## 데이터 concat
- concat필요하신 분들만 진행해주세요!

In [ ]:
import pandas as pd

keyword = ['키워드1', '키워드2', '키워드3', '키워드4', '키워드5']
data_list = []  # 데이터를 임시로 저장할 리스트

# 각 키워드에 해당하는 데이터를 불러와서 리스트에 추가
for i in keyword:
    data = pd.read_csv(f"네이버포털_카테고리분류명_이름_{i}.csv")
    data_list.append(data)

# 리스트를 한 번에 합치기
df = pd.concat(data_list, axis=0).reset_index(drop=True)

df

## 데이터 통합본 불러오기
- 통합본이 따로 있는 경우!

In [ ]:
df1 = pd.read_csv("네이버포털_공간_다운_추가.csv")
df2 = pd.read_csv("네이버포털_공간_다운_통합.csv")
df = pd.concat([df1, df2], axis =0)
print(df.shape)
df.head()

## 중복제거

In [ ]:
# 중복 제거 및 인덱스 초기화
df = df.drop_duplicates(subset=['date', 'title', 'contents', 'comments'], keep='first', inplace=False)
df = df.reset_index(drop=True)

# 중복 제거 후 데이터프레임 길이 출력
df.head()

In [ ]:
df.shape

## 날짜 기준 필터링

In [ ]:
# 날짜 변환 함수 정의
def convert_to_date_only(date_str):
    try:
        # 포맷 없이 변환 시도
        return pd.to_datetime(date_str).date()
    except Exception:
        try:
            # 두 번째 형식 처리: "24.12.06."
            return pd.to_datetime(date_str, format='%y.%m.%d.').date()
        except Exception as e:
            print(f"Error converting {date_str}: {e}")
            return pd.NaT  # 변환 실패 시 NaT 반환

# 데이터프레임에 변환 적용
df['date'] = df['date'].apply(convert_to_date_only)

# NaT 값 확인
print("변환 실패한 데이터:")
print(df[df['date'].isna()])

# 유효한 데이터만 남기기
df = df.dropna(subset=['date'])

# 기준 날짜로 필터링
df = df[df['date'] >= pd.to_datetime('2020-01-01').date()].reset_index(drop=True)

# 결과 확인
df

## \n 제거

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'\n', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    return text

df['title'] = df['title'].apply(preprocess_text)
df['contents'] = df['contents'].apply(preprocess_text)
df['comments'] = df['comments'].apply(preprocess_text)
df['date'] = df['date'].apply(preprocess_text)

## 제목에서 광고글 제거

In [ ]:
df1 = df.copy()

words_to_delete = ['협찬', '체험단', '무료체험', '예약링크', '홍보','무료제공','무상제공','위스토어', '출석체크','체험단','긴급','가입인사','모집','양도매매',]

pattern = '|'.join(words_to_delete)

delete = df1['title'].str.contains(pattern)

In [ ]:
#삭제할 행 확인
df1['title'][df1.index[delete]]

In [ ]:
df1.drop(df1[delete].index, inplace = True)
df2= df1.reset_index(drop=True)
df2

## 10글자 미만 댓글 제거

In [ ]:
# 댓글 리스트에서 10글자 미만의 요소를 제거하는 함수
def filter_short_comments(comment_list):
    # 리스트인지 확인 후 각 요소의 길이를 확인해 10글자 이상인 요소만 남김
    if isinstance(comment_list, list):
        return [comment for comment in comment_list if len(comment) >= 10]
    return comment_list  # 리스트가 아니면 그대로 반환

# DataFrame의 comments 열에 적용
df2['comments'] = df2['comments'].apply(filter_short_comments)

# comments 열이 빈 리스트인 행 제거
df2['comments'] = df2['comments'].apply(lambda x: '댓글 없음' if isinstance(x, list) and len(x) == 0 else ' '.join(x) if isinstance(x, list) else str(x))

df2

## 본문 10글자 미만 제거

In [ ]:
# 10글자 미만인 행 제거
df2 = df2[df2['contents'].str.len() >= 10]

# 결과 출력
df2

In [ ]:
df3 = df2.copy()

In [ ]:
df3['contents'] = df3['contents'].str.replace(r'\[이미지[^\]]*\]', '', regex=True)
df3['contents'] = df['contents'].str.replace(r'\s+', ' ', regex=True)

df3['comments'] = df3['comments'].str.replace(r'\[이미지[^\]]*\]', '', regex=True)
df3['comments'] = df3['comments'].str.replace(r'\s+', ' ', regex=True)
# 결과 확인
df3

## 제목, 본문, 댓글을 하나로 통합

In [ ]:
# 혹시 리뷰 중에 Null 값을 가진 샘플이 있는지 확인
print(df3.isnull().sum())

In [ ]:
df3 = df3.dropna()

In [ ]:
# 위에서 Null값이 나오면 공백으로 수정 (Null 값 포함해서 doc로 합치면 Null값으로 반환됨)
df3['contents'] = df3['contents'].fillna('')

print(df3.isnull().values.any()) # 제거되었는지 확인

In [ ]:
df3.loc[:,'review'] = df3['title'] + ' ' + df3['contents'] + ' ' + df3['comments']
df3['review']
df3 = df3.reset_index(drop=True)

In [ ]:
df_doc = df3.review
df_doc

## 반복되는 문장 삭제

In [ ]:
# 따옴표 안에 삭제할 문구를 복사해서 붙여넣기 해준다.
df_doc = df_doc.str.replace("경상도 3pl 전문 라온아토입니다","")
df_doc = df_doc.str.replace("(원스톱 사업자등록) 월 18000원 비상주 서비스 바로 가입! 바로 임대차계약서 발급까지!","")
df_doc = df_doc.str.replace("출석합니다","")
df_doc = df_doc.str.replace("출첵합니다.","")
df_doc = df_doc.str.replace("가입인사","")
df_doc = df_doc.str.replace("잘부탁드립니다.","")
df_doc = df_doc.str.replace("감사합니다.","")
df_doc = df_doc.str.replace("부탁드립니다","")
df_doc = df_doc.str.replace("연락부탁드립니다.","")

df_doc = df_doc.str.replace("긴글+ 사진폭탄 주의","")
df_doc = df_doc.str.replace("질문 있습니당.","")
df_doc = df_doc.str.replace("★","")

df_doc = df_doc.str.replace("댓글 없음","")
df_doc = df_doc.str.replace("< 아래 양식을 지켜주셔야 글 삭제와 회원강등이 안되니 꼭 지켜서 작성해주세요 >","")
df_doc = df_doc.str.replace("양식을 무시할 경우 운영규칙상 삭제 및 활동정지 될수 있으니 참고부탁드립니다.","")
df_doc = df_doc.str.replace("인스타 양도글은 인터넷 진흥원에서 안된다고 경고를 해서 카페에서 금지하고 있습니다","")
df_doc = df_doc.str.replace("카페활동 없이 양도글만 남길 경우 사이트의 자세한 설명없이 개인연락 유도글은 제재하고 있습니다","")

df_doc = df_doc.str.replace("안녕하세요","")
df_doc = df_doc.str.replace("안녕하세요!","")
df_doc = df_doc.str.replace("안녕하세요.","")

df_doc = df_doc.str.replace("쪽지드렸습니다", "")
df_doc = df_doc.str.replace("챗드릴게요","")
df_doc = df_doc.str.replace("쪽지주세요","")
df_doc = df_doc.str.replace("쪽지드릴게요","")
df_doc = df_doc.str.replace("쪽지 드렸어요","")
df_doc = df_doc.str.replace("댓글이나 쪽지로 주세요","")

df_doc = df_doc.str.replace(" 카페 활동 공지 사항 필독 <-- 가입인사 글쓰기 후 확인하시고 활동하셔야 합니다","")
df_doc = df_doc.str.replace("◆◆◆◆◆◆    진단평가와 관련된 내용만 올려주세요 그 외의 글은 삭제 및 활동제재합니다 ◆◆◆◆◆","")
df_doc = df_doc.str.replace("구합니다.","")
df_doc = df_doc.str.replace("등업조건:가입인사,출석 게시판 각 1개씩 글을 쓰고 각 게시판에 10씩 댓글을 달면 자동등업","")
df_doc = df_doc.str.replace("판매합니다","")
df_doc = df_doc.str.replace("수고하세요.","")
df_doc = df_doc.str.replace("쪽지 부탁드려요.","")
df_doc = df_doc.str.replace("소통해요.","")
df_doc = df_doc.str.replace("소통하고싶어요.","")
df_doc = df_doc.str.replace("해드리겠습니다.","")
df_doc = df_doc.str.replace("진행해드리겠습니다.","")
df_doc = df_doc.str.replace("말씀드리겠습니다.","")
df_doc = df_doc.str.replace("모집중에있습니다.","")
df_doc = df_doc.str.replace("양도합니다.","")
df_doc = df_doc.str.replace("전매합니다.","")
df_doc = df_doc.str.replace("확인합니다","")

df_doc = df_doc.str.replace("댓글없음.","")
df_doc = df_doc.str.replace("나눔후기\)","")
df_doc = df_doc.str.replace("\[체험후기\]","")

In [ ]:
df_doc

## 텍스트 전처리

In [ ]:
df_doc = df_doc.str.replace("[0-9]+갤","개월", regex = True) # ex.8갤->개월(맞춤법 검사 시 교정 안 됌)
df_doc = df_doc.str.replace("[^가-힣a-zA-z ]"," ", regex=True) # 한글,영어만 남기고 특수문자, 숫자 삭제
df_doc = df_doc.str.replace("\s+", " ", regex = True) # white space 제거
df_doc = df_doc.str.replace(r"\\n", " ", regex=True)
df_doc = df_doc.str.replace(r'[^\w\s가-힣]', '', regex=True)

df_doc = df_doc.str.strip()

In [ ]:
df_doc

In [ ]:
doc = pd.DataFrame(df_doc)
doc

In [ ]:
doc['review'] = doc['review'].str.replace(r'\[이미지[^\]]*\]', '', regex=True)
doc['review'] = doc['review'].str.replace(r'\s+', ' ', regex=True)
# 결과 확인
doc

In [ ]:
doc.to_csv("네이버포털_전처리완료.csv", index=False, encoding='utf-8-sig')